In [7]:
# Installation of Required Libraries
!pip install transformers datasets torch

In [8]:
#import os

#os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"
#os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"]="0.7"

In [9]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the IMDB dataset
dataset = load_dataset('imdb')

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the dataset
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

# Load the BERT model
model = BertForSequenceClassification.from_pretrained('bert-large-uncased')

# Define metrics
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Training arguments with optimizations
training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",     
    save_strategy="epoch",           
    save_total_limit=2,              
    num_train_epochs=3,              
    per_device_train_batch_size=2,  
    per_device_eval_batch_size=4,   
    gradient_accumulation_steps=16,  
    warmup_steps=500,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    learning_rate=2e-5,  # Optimized learning rate
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Set to eval_loss to ensure the best generalization
    fp16=False,  # Disable mixed precision
    # Remove bf16 to avoid compatibility issues on MacBook Pro M3
    disable_tqdm=False,
    report_to=[]
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                        
    args=training_args,                 
    train_dataset=train_dataset,         
    eval_dataset=test_dataset,            
    compute_metrics=compute_metrics      
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate(test_dataset)
print(results)

# Sample Inference
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    return torch.argmax(probs).item()

# Test inference
sample_text = "I absolutely loved this movie! The performances were stellar."
print("Sentiment:", "Positive" if predict_sentiment(sample_text) == 1 else "Negative")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.196300,0.156451,0.940160,0.939281,0.953287,0.925680
1,0.189200,0.152401,0.945400,0.944078,0.967504,0.921760
2,0.080300,0.195632,0.951320,0.951804,0.942436,0.961360


{'eval_loss': 0.15240108966827393, 'eval_accuracy': 0.9454, 'eval_f1': 0.9440780040149125, 'eval_precision': 0.9675035687295322, 'eval_recall': 0.92176, 'eval_runtime': 2775.1286, 'eval_samples_per_second': 9.009, 'eval_steps_per_second': 2.252, 'epoch': 2.99904}


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [10]:
# Check if MPS is available and set the device
device = torch.device("mps") if torch.has_mps else torch.device("cpu")
print("Device: ", device)

Device:  mps


/var/folders/51/ygbp8z0j7r57_z285c01c66w0000gn/T/ipykernel_93873/15265501.py:2: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps") if torch.has_mps else torch.device("cpu")


In [14]:
# Path to the directory where the fine-tuned model is saved
#model_path = 'fine-tuned-bert'
model_path = '/Users/jigarpatel/LLMs/Experiments/FineTuneLLMSentimentAnalysisTwitterSentimentDataset101/FineTuneTwitterSentiment/results/checkpoint-2343'

# Load the fine-tuned BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

# Load the fine-tuned BERT model and move it to the appropriate device
model = BertForSequenceClassification.from_pretrained(model_path).to(device)

OSError: Can't load tokenizer for '/Users/jigarpatel/LLMs/Experiments/FineTuneLLMSentimentAnalysisTwitterSentimentDataset101/FineTuneTwitterSentiment/results/checkpoint-2343'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/Users/jigarpatel/LLMs/Experiments/FineTuneLLMSentimentAnalysisTwitterSentimentDataset101/FineTuneTwitterSentiment/results/checkpoint-2343' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [12]:
# Sample Inference
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(device)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    return torch.argmax(probs).item()


In [13]:
# Test inference
sample_text = "I absolutely loved this movie! The performances were stellar."
print("Sentiment:", "Positive" if predict_sentiment(sample_text) == 1 else "Negative")

Sentiment: Negative
